<a href="https://colab.research.google.com/github/KlimachyovDaniil/Parametric-Complexity/blob/main/L1_determinated_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
'''
Класс линейного множителя
Первое поле хранит номера ненулевых переменных и их коээфициент
Второе поле хранит свободный член, поскольку при подстановке
конкретного значения могут появится множитель вида x_i - c '''
class Linear_multiplier:
    def __init__(self, list_of_coefficients):
        self.non_zeros_coefficions = {ind:coef for ind, coef in enumerate(list_of_coefficients) if coef != 0}
        self.const = 0


'''Функция для поиска запретов и нахождения наименьшего элемента'''
def eliminate_variable(list_of_multipliers, x_i):
  m = len(list_of_multipliers)
  list_of_restrictions = set()
  for multiplier in list_of_multipliers:
    if len(multiplier.non_zeros_coefficions) == 1:
      if multiplier.non_zeros_coefficions.get(x_i, 'skip') != 'skip':
        list_of_restrictions.add(multiplier.const/multiplier.non_zeros_coefficions[x_i])
  allowed_elements = set(range(-m, m+1))
  for elem in list_of_restrictions:
    allowed_elements.discard(elem)
  min_elem = m
  for i in allowed_elements:
    if abs(i) < abs(min_elem):
      min_elem = i
  for multiplier in list_of_multipliers:
    if multiplier.non_zeros_coefficions.get(x_i, 'skip') != 'skip':
      multiplier.const += -min_elem * multiplier.non_zeros_coefficions[x_i]
      del multiplier.non_zeros_coefficions[x_i]
  return min_elem


'''Функция для построения вектора'''
def determinated_l1_algo_v2(list_of_multipliers):
  m, n = len(list_of_multipliers), len(list_of_multipliers[0])
  vectors = []
  for i in range(m):
    vectors.append(Linear_multiplier(list_of_multipliers[i]))
  vector = [0] * n
  for x_i in range(n):
    vector[x_i] = eliminate_variable(vectors, x_i)
  return vector

n, m = 3, 80
vectors = np.random.randint(-2,2, size=(m,n))
answer = determinated_l1_algo_v2(vectors)

In [3]:
def test(n, m, num_tests=1000):
  for i in range(num_tests):
    while True:
      vectors = np.random.randint(-m,m+1, size=(m,n))
      vectors = vectors[~np.all(vectors==0, axis=1)]
      if len(vectors) == m:
        break
    vector = determinated_l1_algo_v2(vectors)
    if np.any(np.isclose(np.dot(vectors, vector),0)):
      print('Not passed')
      break
  print('All tests passed')

In [ ]:
test(1000,5)
test(10,1000)
test(1,1)

All tests passed


In [ ]:
def generate_vector(n, M):
    """Генерация случайного целочисленного вектора размерности n с ограничением нормы M."""
    return np.random.randint(-M, M+1, size=n)


def is_non_orthogonal(v, vectors):
  """Проверка, что точка v не попадает ни на одну из гиперплоскостей"""
  if np.any(np.isclose(np.dot(vectors, v),0)):
    return False
  else:
    return True

def rand_l1_algo(n, vectors, M, max_attempts=10):
    """Поиск целочисленного вектора, не ортогонального ни одному вектору из списка, ограниченного нормой M."""
    for _ in range(max_attempts):
        v = sample_point(n, M)
        if is_non_orthogonal(v, vectors):
          return v

# Функция для измерения времени поиска
def measure_time(algorithm, n, vectors, M, max_attempts=10):
    if algorithm == 'rand_l1':
      start_time = perf_counter()
      v = rand_l1_algo(n, vectors, M, max_attempts=max_attempts)
      process_time = perf_counter() - start_time
      if v is not None:
        norm = np.max(np.abs(v))
      else:
        norm = M
      return process_time, norm
    if algorithm == 'det_l1':
      start_time = perf_counter()
      v = determinated_l1_algo_v2(vectors)
      process_time = perf_counter() - start_time
      if v is not None:
        norm = np.max(np.abs(v))
      else:
        norm = M
      return process_time, norm